In [14]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
import numpy as np

# 1. Chargement des données
file_path = "Programmes_avec_budget_et_coordonnees_v2.csv"
df = pd.read_csv(file_path)

# 2. Préparation des données (Budget et Symboles)
# On remplace les NaN par 0 pour le traitement
df['Budget_Estime'] = df['Budget_Estime'].fillna(0)

# Définition du statut pour le symbole : "Budget Connu" vs "Non Chiffré"
df['Statut_Budget'] = df['Budget_Estime'].apply(lambda x: 'Budget Connu' if x > 0 else 'Non Chiffré')
df['Symbole_Point'] = df['Statut_Budget'].map({'Budget Connu': 'circle', 'Non Chiffré': 'x'})

# Calcul de la taille pour avoir plus de variation
# On utilise une fonction puissance (x^0.35) plutôt que log.
# Cela accentue les écarts entre millions et milliards tout en restant affichable.
def calculate_size(budget):
    if budget <= 0:
        return 5 # Taille fixe pour les croix
    else:
        # Formule de puissance ajustée pour l'échelle
        return np.power(budget, 0.35)

df['Taille_Calculee'] = df['Budget_Estime'].apply(calculate_size)

# Normalisation de la taille des bulles (uniquement pour les budgets connus) pour qu'elles soient entre 8 et 60 px
max_val = df.loc[df['Budget_Estime'] > 0, 'Taille_Calculee'].max()
min_val = df.loc[df['Budget_Estime'] > 0, 'Taille_Calculee'].min()

def normalize_size(row):
    if row['Budget_Estime'] <= 0:
        return 4 # Petite taille pour les croix
    else:
        # Normalisation min-max projetée sur 8-60 pixels
        val = row['Taille_Calculee']
        return 8 + (val - min_val) * (60 - 8) / (max_val - min_val)

df['Taille_Finale'] = df.apply(normalize_size, axis=1)


# 3. Gestion du chevauchement (Jittering)
# Ajout d'un bruit aléatoire pour décaler les points superposés
np.random.seed(42)
noise_strength = 0.25 # Décalage modéré

df['X_visu'] = df['X'] + np.random.uniform(-noise_strength, noise_strength, size=len(df))
df['Y_visu'] = df['Y'] + np.random.uniform(-noise_strength, noise_strength, size=len(df))
df['Z_visu'] = df['Z'] + np.random.uniform(-noise_strength, noise_strength, size=len(df))

# Bornage pour rester dans le cube 0-10
df[['X_visu', 'Y_visu', 'Z_visu']] = df[['X_visu', 'Y_visu', 'Z_visu']].clip(0, 10)


# 4. Création du graphique 3D
# Nous utilisons ici graph_objects pour un contrôle fin des symboles
fig = go.Figure()

# Boucle pour ajouter deux traces : une pour les bulles, une pour les croix
# Cela permet d'avoir une légende propre
for statut, symbole in [('Budget Connu', 'circle'), ('Non Chiffré', 'x')]:

    df_subset = df[df['Statut_Budget'] == statut]

    fig.add_trace(go.Scatter3d(
        x=df_subset['X_visu'],
        y=df_subset['Y_visu'],
        z=df_subset['Z_visu'],
        mode='markers',
        name=statut,
        marker=dict(
            size=df_subset['Taille_Finale'],
            symbol=symbole,
            color=df_subset['Y'], # Couleur selon l'axe Stratégie
            colorscale='Viridis',
            opacity=0.8,
            line=dict(width=0) # Pas de contour pour les bulles pour faire plus propre
        ),
        # Informations au survol
        text=df_subset["Programme d'intervention"], # Nom affiché au survol
        customdata=np.stack((
            df_subset['Budget_Description'],
            df_subset['Budget_Estime'],
            df_subset['X'], df_subset['Y'], df_subset['Z']
        ), axis=-1),
        hovertemplate=(
            "<b>%{text}</b><br><br>" +
            "Budget: %{customdata[0]}<br>" +
            "Valeur: %{customdata[1]:,.0f} €<br>" +
            "X (Territoire): %{customdata[2]}<br>" +
            "Y (Stratégie): %{customdata[3]}<br>" +
            "Z (Levier): %{customdata[4]}<extra></extra>"
        )
    ))

# 5. Mise en page et axes
fig.update_layout(
    title="Cartographie 3D des Programmes Publics (Bulles = Budget / Croix = Inconnu)",
    scene=dict(
        xaxis_title='Territoire (0-Rural -> 10-Urbain)',
        yaxis_title='Stratégie (0-Social -> 10-Éco)',
        zaxis_title='Levier (0-Soft -> 10-Hard)',
        xaxis=dict(range=[0, 10], backgroundcolor="rgb(240, 240, 240)"),
        yaxis=dict(range=[0, 10], backgroundcolor="rgb(240, 240, 240)"),
        zaxis=dict(range=[0, 10], backgroundcolor="rgb(240, 240, 240)"),
        aspectmode='cube'
    ),
    margin=dict(l=0, r=0, b=0, t=40),
    legend=dict(yanchor="top", y=0.9, xanchor="left", x=0.05)
)

fig.show()